# 🎯 RIQ MACHINE - NBA Prediction & Analysis

## Features:
- ✅ Load trained neural hybrid models (TabNet + LightGBM)
- ✅ Fetch live NBA games and player stats
- ✅ Generate predictions with full feature engineering
- ✅ Compare to betting lines (find value)
- ✅ Backtest on historical data
- ✅ Track performance and ROI

## Prerequisites:
1. Trained models from `NBA_COLAB_SIMPLE.ipynb`
2. Upload `nba_models_trained.zip` to this Colab
3. Upload `priors_data.zip` (for feature engineering)
4. Upload `PlayerStatistics.csv` (for backtest data)

## Quick Start:
1. Run Setup cell
2. Upload model files + data
3. Choose: Backtest OR Live Predictions

---

## 📦 Setup & Installation

In [ ]:
# ============================================================
# SETUP - Install packages and download code
# ============================================================

print("📦 Installing packages...")
!pip install -q nba-api pytorch-tabnet lightgbm scikit-learn pandas numpy requests

print("\n📥 Downloading training code (for feature engineering)...")
import os
import shutil

os.chdir('/content')

# Remove old code if exists
if os.path.exists('meep'):
    shutil.rmtree('meep')
    print("🧹 Cleaned up old code")

!git clone https://github.com/tyriqmiles0529-pixel/meep.git
os.chdir('meep')

print("\n📍 Code version:")
!git log -1 --oneline

# Add to Python path so we can import train_auto
import sys
sys.path.insert(0, '/content/meep')

print("\n✅ Setup complete!")
print("\nNext: Run 'Upload Models & Data' cell")

---

## 📤 Upload Models & Data

In [ ]:
# ============================================================
# UPLOAD - Models and priors data
# ============================================================

from google.colab import files
import os

os.chdir('/content')

print("="*70)
print("📤 UPLOAD REQUIRED")
print("="*70)
print("\nYou need 2 files:")
print("  1. nba_models_trained.zip (your trained models)")
print("  2. priors_data.zip (for feature engineering)")
print("\nUploading...\n")

uploaded = files.upload()

print("\n📦 Extracting files...")

# Extract models
if os.path.exists('nba_models_trained.zip'):
    !unzip -q nba_models_trained.zip
    !rm nba_models_trained.zip
    print("✅ Models extracted to ./models/ and ./model_cache/")
else:
    print("⚠️  nba_models_trained.zip not found")

# Extract priors
if os.path.exists('priors_data.zip'):
    !unzip -q priors_data.zip
    !rm priors_data.zip
    print("✅ Priors extracted to ./priors_data/")
else:
    print("⚠️  priors_data.zip not found")

# Verify
print("\n🔍 Verification:")
if os.path.exists('models'):
    models = [f for f in os.listdir('models') if f.endswith('.pkl')]
    print(f"   ✅ Found {len(models)} model files")
    for m in models:
        print(f"      - {m}")
else:
    print("   ❌ models/ directory not found")

if os.path.exists('priors_data'):
    priors = [f for f in os.listdir('priors_data') if f.endswith('.csv')]
    print(f"   ✅ Found {len(priors)} priors files")
else:
    print("   ❌ priors_data/ directory not found")

print("\n✅ Upload complete!")
print("\nNext: Choose Backtest OR Live Predictions")

---

## 🧪 Option A: Backtest (Test on Historical Data)

In [ ]:
# ============================================================
# BACKTEST - Test models on recent historical games
# ============================================================

import pickle
import pandas as pd
import numpy as np
from pathlib import Path

print("="*70)
print("🧪 BACKTEST MODE")
print("="*70)

# Load trained models
print("\n📊 Loading models...")
models = {}
props = ['minutes', 'points', 'rebounds', 'assists', 'threes']

for prop in props:
    model_path = Path(f'models/{prop}_model.pkl')
    if model_path.exists():
        with open(model_path, 'rb') as f:
            models[prop] = pickle.load(f)
        print(f"   ✅ {prop.capitalize()}: {len(models[prop].feature_names)} features")
    else:
        print(f"   ❌ {prop.capitalize()}: Not found")

print(f"\n✅ Loaded {len(models)} models")

# Check model structure
print("\n🔍 Model Structure (Points):")
points_model = models.get('points')
if points_model:
    print(f"   TabNet: {'✅' if hasattr(points_model, 'tabnet') else '❌'}")
    print(f"   LightGBM: {'✅' if hasattr(points_model, 'lgbm') else '❌'}")
    print(f"   Embedding scaler: {'✅' if hasattr(points_model, 'embedding_scaler') else '❌'}")
    print(f"   Total features: {len(points_model.feature_names)}")
    
    # Show feature importance
    if hasattr(points_model, 'lgbm') and hasattr(points_model.lgbm, 'feature_importances_'):
        importances = sorted(
            zip(points_model.feature_names + [f'tabnet_emb_{i}' for i in range(24)], 
                points_model.lgbm.feature_importances_),
            key=lambda x: x[1],
            reverse=True
        )[:10]
        
        print("\n🧠 Top 10 Most Important Features:")
        for feat, imp in importances:
            emoji = "🧠" if 'tabnet_emb' in feat else "📊"
            print(f"   {emoji} {feat}: {imp:.1%}")

print("\n" + "="*70)
print("📌 TODO: Add backtest logic")
print("="*70)
print("\nNext steps:")
print("1. Load validation data from training")
print("2. Generate predictions")
print("3. Compare to actual results")
print("4. Calculate RMSE, MAE, directional accuracy")

---

## 🎯 Option B: Live Predictions (Today's Games)

In [ ]:
# ============================================================
# LIVE PREDICTIONS - Fetch today's games and predict
# ============================================================

from datetime import datetime
from nba_api.stats.endpoints import ScoreboardV2
import pickle
from pathlib import Path

print("="*70)
print("🎯 LIVE PREDICTION MODE")
print("="*70)

# Get today's date
today = datetime.now().strftime('%Y-%m-%d')
print(f"\n📅 Date: {today}")

# Fetch today's games
print("\n🏀 Fetching today's NBA games...")
try:
    scoreboard = ScoreboardV2(game_date=today)
    games = scoreboard.get_data_frames()[0]
    
    if len(games) == 0:
        print("   ⚠️  No games scheduled for today")
    else:
        print(f"   ✅ Found {len(games)} games")
        print("\n📋 Games:")
        for idx, game in games.iterrows():
            home = game.get('HOME_TEAM_ABBREVIATION', 'HOME')
            away = game.get('VISITOR_TEAM_ABBREVIATION', 'AWAY')
            print(f"      {away} @ {home}")
except Exception as e:
    print(f"   ❌ Error fetching games: {e}")
    games = None

# Load models
print("\n📊 Loading models...")
models = {}
props = ['minutes', 'points', 'rebounds', 'assists', 'threes']

for prop in props:
    model_path = Path(f'models/{prop}_model.pkl')
    if model_path.exists():
        with open(model_path, 'rb') as f:
            models[prop] = pickle.load(f)
        print(f"   ✅ {prop.capitalize()}")

print(f"\n✅ Loaded {len(models)} models")

print("\n" + "="*70)
print("📌 TODO: Add live prediction logic")
print("="*70)
print("\nNext steps:")
print("1. For each game, get probable starters")
print("2. Fetch recent game logs (last 10 games)")
print("3. Use train_auto.py feature engineering")
print("4. Generate predictions")
print("5. Fetch betting lines")
print("6. Find value bets (model vs line difference > threshold)")

---

## 🔧 Helper Functions (Feature Engineering)

In [ ]:
# ============================================================
# FEATURE ENGINEERING - Reuse training code
# ============================================================

# Import feature engineering from train_auto.py
# This ensures features match training exactly!

import sys
sys.path.insert(0, '/content/meep')

try:
    from train_auto import build_players_from_playerstats
    print("✅ Imported feature engineering from train_auto.py")
    print("   This ensures features match training schema exactly!")
except ImportError as e:
    print(f"❌ Failed to import: {e}")
    print("   Make sure you ran the Setup cell first")

# TODO: Add helper functions for:
# - Fetching player recent games
# - Converting API data to training schema
# - Generating predictions
# - Comparing to betting lines

---

## 📊 Evaluation & Performance Tracking

In [ ]:
# ============================================================
# EVALUATION - Track accuracy and ROI
# ============================================================

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_predictions(y_true, y_pred, prop_name):
    """
    Calculate prediction accuracy metrics.
    
    Args:
        y_true: Actual values
        y_pred: Predicted values
        prop_name: Property being predicted (points, rebounds, etc.)
    
    Returns:
        Dict of metrics
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    # Directional accuracy (for betting)
    # TODO: Compare to betting lines
    
    print(f"\n📊 {prop_name.upper()} Performance:")
    print(f"   RMSE: {rmse:.3f}")
    print(f"   MAE: {mae:.3f}")
    
    return {
        'rmse': rmse,
        'mae': mae,
        'prop': prop_name
    }

# TODO: Add betting performance tracking
# - Win rate (over/under)
# - ROI calculation
# - Kelly criterion sizing
# - Bankroll tracking

---

## 💰 Betting Analysis (Value Detection)

In [ ]:
# ============================================================
# BETTING ANALYSIS - Find value bets
# ============================================================

def find_value_bets(predictions, lines, threshold=2.0):
    """
    Find value bets where model disagrees with betting line.
    
    Args:
        predictions: Dict of player predictions {player_name: {prop: prediction}}
        lines: Dict of betting lines {player_name: {prop: line}}
        threshold: Minimum difference to consider (default: 2.0)
    
    Returns:
        List of value bets sorted by edge
    """
    value_bets = []
    
    for player, preds in predictions.items():
        if player not in lines:
            continue
        
        player_lines = lines[player]
        
        for prop, pred_value in preds.items():
            if prop not in player_lines:
                continue
            
            line = player_lines[prop]
            edge = abs(pred_value - line)
            
            if edge >= threshold:
                direction = 'OVER' if pred_value > line else 'UNDER'
                
                value_bets.append({
                    'player': player,
                    'prop': prop,
                    'prediction': pred_value,
                    'line': line,
                    'edge': edge,
                    'direction': direction,
                    'confidence': edge / threshold  # Simple confidence metric
                })
    
    # Sort by edge (highest first)
    value_bets.sort(key=lambda x: x['edge'], reverse=True)
    
    return value_bets

# TODO: Add Kelly criterion bet sizing
# TODO: Add odds fetching (API integration)
# TODO: Add historical bet tracking

---

## 📝 Notes & Documentation

### Model Details:
- **Architecture:** Neural Hybrid (TabNet + LightGBM)
- **Features:** ~218 raw features + 24 TabNet embeddings = 242 total
- **Training Data:** 1.6M+ player-games (1974-2025)
- **Expected Accuracy:** Points MAE ~3.0 (±3 points)

### Feature Engineering:
- Phase 1-7: All phases from training
- Basketball Reference priors: ~68 advanced stats
- Momentum features: Short/medium/long-term trends
- Situational context: Rest, schedule density, opponent history

### Betting Strategy:
- Find value when |prediction - line| > threshold (default: 2.0)
- Need 52.4% accuracy to beat -110 vig
- Target: 60-65% accuracy on value bets
- Expected ROI: +15-25%

### TODO:
1. Implement backtest logic
2. Implement live prediction pipeline
3. Add betting line fetching (API)
4. Add performance tracking
5. Add Kelly criterion bet sizing